In [ ]:
# gh repo clone abudesai/timeVAE

# Import packages

In [1]:
import sys

sys.path.insert(0, 'D:/Market_Scenario_Generator_Hackathon')

In [2]:

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)
from scipy.stats import levy_stable

from datetime import datetime
from scipy.stats import kstest
from scipy.stats import jarque_bera
from arch.unitroot import ADF
from scipy.stats import kurtosis
from scipy.stats import skew
from arch import arch_model

from sklearn.metrics import mean_squared_error as mse

import pickle

import torch
import torch.nn as nn

import matplotlib.pyplot as plt
plt.style.use('classic')


# %config inlinebackend.figure_format = 'svg'

In [3]:
import ml_collections
import copy
import yaml
from tqdm import tqdm
import os
import torch
import torch.nn as nn
from os import path as pt
import pickle
from torch.utils.data import DataLoader, TensorDataset
from src.baselines.base import BaseTrainer
from src.evaluation.summary import full_evaluation
from src.utils import get_experiment_dir, set_seed, init_weights, save_obj, load_obj
from src.evaluation.scores import compute_auc

In [4]:
from IPython import display

import glob
import imageio
import PIL
import tensorflow_probability as tfp
import time

In [5]:
import tensorflow as tf
import keras
from keras import layers

In [6]:
a = torch.tensor(np.random.normal(size = (1000, 5, 10))).to(torch.float32)
b = torch.tensor(np.random.normal(size = (1000, 5, 10))).to(torch.float32)


In [7]:
config_dir = 'src/evaluation/config.yaml'
with open(config_dir) as file:
    eval_config = ml_collections.ConfigDict(yaml.safe_load(file))

eval_config.device = 'cpu'
eval_config.Evaluation.n_eval = 1
eval_config.Evaluation.metrics_enabled = [
    # 'hist_loss',
    # 'discriminative_score',
    'predictive_score',
    # 'acf_loss',
    # 'cov_loss',
    # 'cross_corr',
    # 'var',
    # 'es'
    ]

In [37]:
eval_config

Evaluation:
  TestMetrics:
    acf_loss:
      keep_init: false
      stationary: false
    cov_loss: None
    cross_corr: None
    discriminative_score:
      batch_size: 128
      epochs: 50
      hidden_size: 32
      num_layers: 2
    es:
      alpha: 0.05
      dims:
      - 0
      - 2
      - 4
      - 6
      - 8
    hist_loss:
      keep_init: true
      n_bins: 30
    predictive_score:
      batch_size: 128
      epochs: 10
      hidden_size: 32
      num_layers: 2
    var:
      alpha: 0.05
      dims:
      - 0
      - 2
      - 4
      - 6
      - 8
  batch_size: 2
  metrics_enabled:
  - predictive_score
  n_eval: 1
  test_ratio: 0.5
batch_size: 16
device: cpu
num_workers: 0
seed: 0
ts_length: 5

In [9]:
a.shape, b.shape

(torch.Size([1000, 5, 10]), torch.Size([1000, 5, 10]))

In [35]:
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, out_size=2):
        super(Discriminator, self).__init__()
        self.rnn = nn.GRU(input_size=input_size, num_layers=num_layers,
                            hidden_size=hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, out_size)

    def forward(self, x):
        x = self.rnn(x)[0][:, -1]
        return self.linear(x)
    
class Predictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, out_size):
        super(Predictor, self).__init__()
        self.rnn = nn.LSTM(input_size=input_size, num_layers=num_layers,
                            hidden_size=hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, out_size)

    def forward(self, x):
        x = self.rnn(x)[0][:, -1]
        return self.linear(x)

In [44]:
def create_dl(dl1, dl2, batch_size, cutoff=False):
    x, y = [], []

    if cutoff:
        _, T, C = next(iter(dl1))[0].shape
        T_cutoff = T# int(T / 10)
        for data in dl1:
            x.append(data[0][:, :-T_cutoff])
            y.append(data[0][:, -T_cutoff:].reshape(data[0].shape[0], -1))
        for data in dl2:
            x.append(data[0][:, :-T_cutoff])
            y.append(data[0][:, -T_cutoff:].reshape(data[0].shape[0], -1))
        x, y = torch.cat(x), torch.cat(y)
    else:
        for data in dl1:
            x.append(data[0])
            y.append(torch.ones(data[0].shape[0], ))
        for data in dl2:
            x.append(data[0])
            y.append(torch.zeros(data[0].shape[0], ))
        x, y = torch.cat(x), torch.cat(y).long()
    idx = torch.randperm(x.shape[0])
    return DataLoader(TensorDataset(x[idx].view(x.size()), y[idx].view(y.size())), batch_size=batch_size)


In [45]:
data = create_dl(a, b, 128, False)

In [46]:
# x, y = [], []
# for data in a:
#     x.append(data[0])
#     y.append(torch.ones(data[0].shape[0], ))
# for data in b:
#     x.append(data[0])
#     y.append(torch.zeros(data[0].shape[0], ))
# x, y = torch.cat(x), torch.cat(y).long()

# idx = torch.randperm(x.shape[0])
# data = DataLoader(TensorDataset(x[idx].view(x.size()), y[idx].view(y.size())), batch_size=128)

In [52]:
Discriminator(data.dataset[0][0].shape[-1], 32, 2)

IndexError: tuple index out of range

In [21]:
x, y = [], []

_, T, C = a.shape
T_cutoff = T# int(T / 10)
for data in a:
    x.append(data[:, :-T_cutoff])
    y.append(data[:, -T_cutoff:].reshape(data.shape[0], -1))
for data in b:
    x.append(data[:, :-T_cutoff])
    y.append(data[:, -T_cutoff:].reshape(data.shape[0], -1))
x, y = torch.cat(x), torch.cat(y)

In [31]:
a[1]

tensor([[-0.0378,  0.1144, -0.7082, -0.9840, -1.8507,  0.4343,  0.9820, -0.5794,
         -1.4352, -1.4566],
        [-0.3020,  1.8506, -0.1144, -0.0046, -0.2261, -0.3068, -0.2322,  0.4271,
          0.7183, -0.9858],
        [ 0.5435,  0.6174,  0.2261,  0.1382, -0.4024,  1.2514,  0.5811,  0.0362,
          0.2440, -1.5786],
        [ 0.0534, -0.7953,  0.6582, -0.6547, -0.5081, -0.3181, -1.2576, -1.4245,
         -0.5771, -1.1078],
        [ 0.0784,  0.3843,  0.6704, -0.7682, -3.0670,  0.2840,  0.2097, -1.1769,
         -0.6799,  1.3006]])

In [28]:
x[0:3]

[tensor([ 0.8091, -0.8005, -0.3658, -0.5525, -0.8998,  0.6143,  0.1107,  0.3867,
         -2.1237,  1.1153]),
 tensor([-0.0378,  0.1144, -0.7082, -0.9840, -1.8507,  0.4343,  0.9820, -0.5794,
         -1.4352, -1.4566]),
 tensor([-1.8947,  0.1736, -0.4496, -0.7401, -1.1643, -1.1178,  2.0437,  0.0021,
         -0.5153, -0.8182])]

In [12]:
x

tensor([[ 0.8091, -0.8005, -0.3658, -0.5525, -0.8998],
        [-0.6538,  0.5717, -0.5840,  0.0976, -0.0611],
        [-1.1906,  0.3701,  0.6433, -1.2294,  1.9542],
        ...,
        [ 1.0284,  0.2927, -1.3365, -0.3138, -0.2156],
        [-1.0111, -0.0097,  0.1510, -0.9441,  0.3544],
        [-0.9989, -1.8588,  0.8027, -0.0116, -0.1134]])

In [13]:
full_evaluation(a, b, eval_config)

 No metrics enabled in group = stylized_fact_scores
---- evaluation metric = predictive_score in group = implicit_scores ----


  0%|          | 0/1 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'shape'

# Data

## Importing

In [ ]:
training_data = pd.read_pickle(f'{path}/Market_Scenario_Generator_Hackathon_starting_kit/data/training_data.pkl')
training_data = training_data[1800:3000, :, 0:10]
training_label = pd.read_pickle(f'{path}/Market_Scenario_Generator_Hackathon_starting_kit/data/training_label.pkl')
training_label = training_label[1800:3000, :]

valid_data = pd.read_pickle(f'{path}/Market_Scenario_Generator_Hackathon_starting_kit/data/valid_data.pkl')
valid_data = valid_data[1800:3000, :, 0:10]
valid_label = pd.read_pickle(f'{path}/Market_Scenario_Generator_Hackathon_starting_kit/data/valid_label.pkl')
valid_label = valid_label[1800:3000, :]

In [ ]:
training_data.shape, training_label.shape, sum(training_label), valid_data.shape, valid_label.shape, sum(valid_label)

In [ ]:
regular_market_data = training_data[(training_label==0).squeeze()]
crisis_market_data = training_data[(training_label==1).squeeze()]

In [ ]:
daily_log_return = training_data[:, :, [0, 2, 4, 6, 8]]
daily_vol = training_data[:, :, [1, 3, 5, 7, 9]]

daily_log_return_1 = training_data[:, 0, [0, 2, 4, 6, 8]]
daily_vol_1 = training_data[:, 0, [1, 3, 5, 7, 9]]

daily_log_return_2 = training_data[:, 1, [0, 2, 4, 6, 8]]
daily_vol_2 = training_data[:, 1, [1, 3, 5, 7, 9]]

daily_log_return_3 = training_data[:, 2, [0, 2, 4, 6, 8]]
daily_vol_3 = training_data[:, 2, [1, 3, 5, 7, 9]]

daily_log_return_4 = training_data[:, 3, [0, 2, 4, 6, 8]]
daily_vol_4 = training_data[:, 3, [1, 3, 5, 7, 9]]

daily_log_return_5 = training_data[:, 4, [0, 2, 4, 6, 8]]
daily_vol_5 = training_data[:, 4, [1, 3, 5, 7, 9]]


In [ ]:
daily_log_return.shape, daily_vol.shape

In [ ]:
ret = daily_log_return[:, 0, 0]
vol = daily_vol[:, 0, 0]

resids = np.ascontiguousarray(ret)

res_model = arch_model(resids,
                       p = 1,
                       q = 1,
                       mean = 'Zero',
                       vol = 'GARCH',
                       dist = 'normal')

GARCH = res_model.fit(update_freq = 0,
                      disp = 'off')

GARCH_vol = GARCH.conditional_volatility

fig, ax = plt.subplots(nrows = 2, ncols = 1, figsize = (20, 10))

ax[0].plot(ret, label = f'RET', color = 'blue', linewidth = 1)

ax[1].plot(vol, label = f'VOLATILITY', color = 'blue', linewidth = 1)
ax[1].plot(GARCH_vol, label = f'GARCH_vol', color = 'green', linewidth = 1)

ax[1].legend(loc = 'upper left')

plt.show()

## Visualization

In [ ]:
fig, ax = plt.subplots(nrows = 5, ncols = 3, figsize = (30, 100))

for i in range(5):
    ax[i, 0].plot(training_data[:, 0, 2*i], label = f'RET', color = 'blue', linewidth = 1)
    ax2 = ax[i, 0].twinx()
    ax2.plot(training_label, color = 'red', alpha = 0.5, linewidth = 2)
    ax2.set_ylim([-0.1, 1.1])
    # ax[i, 0].legend(loc = 'upper right')

    ax[i, 1].plot(training_data[:, 0, 2*i - 1], label = f'VOLATILITY', color = 'blue', linewidth = 1)
    ax2 = ax[i, 1].twinx()
    ax2.plot(training_label, color = 'red', alpha = 0.5, linewidth = 2)
    ax2.set_ylim([-0.1, 1.1])
    # ax[i, 1].legend(loc = 'upper right')

    ax[i, 2].hist(training_data[:, 0, 2*i], bins = 50)


plt.show()

# Model class

## Conditional_VAE

### Define model

In [ ]:
from abc import ABC, abstractmethod

class Sampling(keras.layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class C_VariationalAutoencoder(keras.Model, ABC):
    def __init__(self,
            seq_len,
            feat_dim,
            latent_dim,
            loss_weight,
            **kwargs):
        super(C_VariationalAutoencoder, self).__init__(**kwargs)

        # number of points in one simulated result
        self.seq_len = seq_len

        # number of input features - may be just log daily return or in multivariate case, the number of assets, set to 1
        self.feat_dim = feat_dim

        # number of dimensions in the latent space, set to 1
        self.latent_dim = latent_dim

        self.loss_weight = loss_weight

        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")

        # mse distance between original and generated sample
        self.reconstruction_loss_tracker_1 = keras.metrics.Mean(name="reconstruction_loss_1")

        # probability distance between original and generated sample
        self.reconstruction_loss_tracker_2 = keras.metrics.Mean(name="reconstruction_loss_2")

        # mse between true and generated correlation matrix
        self.reconstruction_loss_tracker_3 = keras.metrics.Mean(name="reconstruction_loss_3")

        # probability distance between the latent space variable Z and the Gaussian distribution
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

        self.encoder = self._get_encoder()
        self.decoder = self._get_decoder()


    def call(self, X):
        X_cur, X_cond = X
        z_mean, _, _ = self.encoder(X_cur)
        x_decoded = self.decoder([z_mean, X_cond])
        if len(x_decoded.shape) == 1:
            x_decoded = x_decoded.reshape((1, -1))
        return x_decoded


    def _get_encoder(self):
        self.encoder_cur_inputs = keras.layers.Input(shape=(self.seq_len, self.feat_dim), name='encoder_curr_input')

        # masked_layer = keras.layers.Masking(mask_value=-1)(self.encoder_inputs)

        x = keras.layers.Flatten()(self.encoder_cur_inputs)
        x = keras.layers.Dense(100, activation = None, name = f'enc_dense_1')(x)

        z_mean = keras.layers.Dense(self.latent_dim, activation = None, name="z_mean")(x)
        z_log_var = keras.layers.Dense(self.latent_dim, activation = None, name="z_log_var")(x)

        encoder_output = Sampling()([z_mean, z_log_var])
        self.encoder_output = encoder_output

        encoder = keras.Model(self.encoder_cur_inputs, [z_mean, z_log_var, encoder_output], name="encoder")
        # encoder.summary()
        return encoder


    def _get_decoder(self):
        decoder_rand_inputs = keras.layers.Input(shape=(self.latent_dim), name='decoder_random')
        decoder_cond_inputs = keras.layers.Input(shape=(1,), name='decoder_cond_input')

        x1 = tf.keras.layers.RepeatVector(100)(decoder_cond_inputs)
        x1 = keras.layers.Flatten()(x1)

        x = keras.layers.concatenate([decoder_rand_inputs, x1])

        x = keras.layers.Dense(100,
                               activation = None,
                               name='decoder_dense',
                               kernel_initializer=keras.initializers.RandomNormal(stddev=0.01),
                               bias_initializer=keras.initializers.Zeros())(x)

        # Lists to store return and volatility layers for each asset
        layers = []

        # Loop to create layers for each asset
        for i in range(1, int(self.feat_dim/2) + 1):
            # Return Layer
            ret_i = keras.layers.Dense(self.seq_len,
                                        activation='tanh',
                                        name=f'ret_{i}',
                                        kernel_initializer=keras.initializers.RandomNormal(stddev=0.01),
                                        bias_initializer=keras.initializers.Zeros())(x)
            ret_i = keras.layers.Reshape(target_shape=(self.seq_len, 1))(ret_i)
            layers.append(ret_i)

            # Volatility Layer
            vol_i = keras.layers.Dense(self.seq_len,
                                        activation='sigmoid',
                                        name=f'vol_{i}',
                                        kernel_initializer=keras.initializers.RandomNormal(stddev=0.01),
                                        bias_initializer=keras.initializers.Zeros())(x)
            vol_i = keras.layers.Reshape(target_shape=(self.seq_len, 1))(vol_i)
            layers.append(vol_i)

        # Concatenating return and volatility layers for all assets
        self.decoder_outputs = keras.layers.concatenate(layers)

        decoder = keras.Model([decoder_rand_inputs, decoder_cond_inputs], self.decoder_outputs, name="decoder")
        # decoder.summary()
        return decoder


    def _get_reconstruction_loss(self, X, X_recons):

        def get_reconst_loss_by_axis(X, X_c, axis):
            x_r = tf.reduce_mean(X, axis = axis)
            x_c_r = tf.reduce_mean(X_recons, axis = axis)
            err = tf.math.squared_difference(x_r, x_c_r)
            loss = tf.reduce_sum(err)
            return loss

        # overall
        err = tf.math.squared_difference(X, X_recons)
        reconst_loss = tf.reduce_sum(err)

        reconst_loss += get_reconst_loss_by_axis(X, X_recons, axis=1)
        reconst_loss += get_reconst_loss_by_axis(X, X_recons, axis=2)

        return reconst_loss


    def summary(self):
        self.encoder.summary()
        self.decoder.summary()


    def train_step(self, X):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(X[0])
            reconstruction = self.decoder([z, X[1]])

            # adjusted MSE between true and generated data across whole, timesteps and features
            reconstruction_loss_1 = self._get_reconstruction_loss(tf.cast(X[0], tf.float64), tf.cast(reconstruction, tf.float64))

            # PSI betweem true and generated data
            reconstruction_loss_2 = tf.keras.losses.KLDivergence()(tf.cast(X[0], tf.float64), tf.cast(reconstruction, tf.float64)) \
                                    + tf.keras.losses.KLDivergence()(tf.cast(reconstruction, tf.float64), tf.cast(X[0], tf.float64))
            ## Reduce by timestep
            reconstruction_loss_2 += tf.keras.losses.KLDivergence()(tf.cast(tf.reduce_mean(X[0], axis = 1), tf.float64),
                                                                    tf.cast(tf.reduce_mean(reconstruction, axis = 1), tf.float64)) \
                                    + tf.keras.losses.KLDivergence()(tf.cast(tf.reduce_mean(reconstruction, axis = 1), tf.float64),
                                                                     tf.cast(tf.reduce_mean(X[0], axis = 1), tf.float64))
            ## Reduce by features
            reconstruction_loss_2 += tf.keras.losses.KLDivergence()(tf.cast(tf.reduce_mean(X[0], axis = 2), tf.float64),
                                                                    tf.cast(tf.reduce_mean(reconstruction, axis = 2), tf.float64)) \
                                    + tf.keras.losses.KLDivergence()(tf.cast(tf.reduce_mean(reconstruction, axis = 2), tf.float64),
                                                                     tf.cast(tf.reduce_mean(X[0], axis = 2), tf.float64))



            # MAE between true and generated correlation matrix
            true_corr = tfp.stats.correlation(X[0], sample_axis = 1, event_axis = -1)
            pred_corr = tfp.stats.correlation(reconstruction, sample_axis = 1, event_axis = -1)

            reconstruction_loss_3 = tf.keras.losses.MeanAbsoluteError()(tf.cast(true_corr, tf.float64), tf.cast(pred_corr, tf.float64))


            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_sum(kl_loss, axis=1)

            total_loss = tf.cast(self.loss_weight[0] * reconstruction_loss_1, tf.float64) \
                            + tf.cast(self.loss_weight[1] * reconstruction_loss_2, tf.float64) \
                            + tf.cast(self.loss_weight[2] * reconstruction_loss_3, tf.float64) \
                            + tf.cast(self.loss_weight[3] * kl_loss, tf.float64)


        grads = tape.gradient(total_loss, self.trainable_weights)

        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker_1.update_state(reconstruction_loss_1)
        self.reconstruction_loss_tracker_2.update_state(reconstruction_loss_2)
        self.reconstruction_loss_tracker_3.update_state(reconstruction_loss_3)

        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss_1": self.reconstruction_loss_tracker_1.result(),
            "reconstruction_loss_2": self.reconstruction_loss_tracker_2.result(),
            "reconstruction_loss_3": self.reconstruction_loss_tracker_3.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

### Train model

In [ ]:

vae = C_VariationalAutoencoder(
    seq_len=5,
    feat_dim = 10,
    latent_dim = 5,
    loss_weight = [3, 2, 2, 1]
)

vae.compile(optimizer=keras.optimizers.Adam(1e-4))


In [ ]:
# vae.summary()

In [ ]:
tf.keras.utils.plot_model(vae.encoder,
                          #to_file = f'{path}/Data/Structure.png',
                          dpi = 96,
                          show_shapes = True,
                          show_dtype = True,
                          show_layer_names = True,
                          show_layer_activations = True,
                          expand_nested=False,
                          show_trainable=False)

In [ ]:
tf.keras.utils.plot_model(vae.decoder,
                          #to_file = f'{path}/Data/Structure.png',
                          dpi = 96,
                          show_shapes = True,
                          show_dtype = True,
                          show_layer_names = True,
                          show_layer_activations = True,
                          expand_nested=False,
                          show_trainable=False)

In [ ]:
batch_size = 32

input_set = tf.data.Dataset.from_tensor_slices((training_data, training_label))
input_set = input_set.batch(batch_size)
dataset = tf.data.Dataset.zip((input_set))
# Initialize the iterator to be passed to the model.fit() function
# data_iter = dataset.make_one_shot_iterator()

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=50)

history = vae.fit(dataset, epochs = 1000, batch_size = batch_size, shuffle = False, verbose = True,
                  callbacks=[callback]
                  )

### Evaluation

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize = (40, 5))

ax[0].plot(history.history['loss'], label = f'TOTAL_LOSS', color = 'red', linewidth = 1.5)
ax[0].legend(loc = 'upper right')

ax[1].plot(history.history['reconstruction_loss_1'], label = f'RECONSTRUCTION_LOSS_1', color = 'blue', linewidth = 1.5)
ax[1].legend(loc = 'upper right')

ax[2].plot(history.history['reconstruction_loss_2'], label = f'RECONSTRUCTION_LOSS_2', color = 'blue', linewidth = 1.5)
ax[2].legend(loc = 'upper right')

ax[3].plot(history.history['reconstruction_loss_3'], label = f'RECONSTRUCTION_LOSS_3', color = 'blue', linewidth = 1.5)
ax[3].legend(loc = 'upper right')

ax[4].plot(history.history['kl_loss'], label = f'kl_LOSS', color = 'orange', linewidth = 1.5)
ax[4].legend(loc = 'upper right')

plt.show()

In [ ]:
random_fac = tf.random.normal(shape = (200, 5))
# random_fac

In [ ]:
regular_conditions = tf.ones(shape = (200, 1))
crisis_conditions = tf.zeros(shape = (200, 1))

In [ ]:
# y_regular_pred = vae.decoder.predict([tf.random.normal(shape = (220, 10)), regular_seed])
# y_crisis_pred = vae.decoder.predict([tf.random.normal(shape = (20, 10)), crisis_seed])

regular_fake_data = vae.decoder.predict([random_fac, regular_conditions])
crisis_fake_data = vae.decoder.predict([random_fac, crisis_conditions])


In [ ]:
config_dir = 'src/evaluation/config.yaml'
with open(config_dir) as file:
    eval_config = ml_collections.ConfigDict(yaml.safe_load(file))


In [ ]:
eval_config.device = 'cpu'
eval_config.Evaluation.n_eval = 100
eval_config.Evaluation.metrics_enabled = ['hist_loss',
                                        #   'discriminative_score',
                                        #   'predictive_score',
                                          'acf_loss',
                                          'cov_loss',
                                          'cross_corr',
                                          'var',
                                          'es']

#### In-sample

In [ ]:
regular_data = training_data[(training_label==0).squeeze()]
crisis_data = training_data[(training_label==1).squeeze()]

In [ ]:
regular_data = torch.tensor(regular_data, dtype = torch.float32)
crisis_data = torch.tensor(crisis_data, dtype = torch.float32)

regular_fake_data = torch.tensor(regular_fake_data)
crisis_fake_data = torch.tensor(crisis_fake_data)

In [ ]:
regular_fake_data.shape, crisis_fake_data.shape, regular_data.shape, crisis_data.shape

In [ ]:

res_dict_regular = full_evaluation(regular_fake_data, regular_data, eval_config)
# for k, v in res_dict_regular.items():
#     print(k, v)

res_dict_crisis = full_evaluation(crisis_fake_data, crisis_data, eval_config)
# for k, v in res_dict_regular.items():
#     print(k, v)

In [ ]:
df_result = pd.DataFrame.from_dict(res_dict_regular, orient='index')
df_result.columns = ['Regular']
df_result['Crisis'] = res_dict_crisis
df_result['Score'] = 0.7*df_result['Crisis'] + 0.3*df_result['Regular']

df_result

In [ ]:
auc = compute_auc(crisis_data, crisis_fake_data, regular_data, regular_fake_data, eval_config)
print(auc)

#### Out-of-sample

In [ ]:
regular_data = valid_data[(valid_label==0).squeeze()]
crisis_data = valid_data[(valid_label==1).squeeze()]

In [ ]:
regular_data = torch.tensor(regular_data, dtype = torch.float32)
crisis_data = torch.tensor(crisis_data, dtype = torch.float32)

regular_fake_data = torch.tensor(regular_fake_data)
crisis_fake_data = torch.tensor(crisis_fake_data)

In [ ]:
regular_fake_data.shape, crisis_fake_data.shape, regular_data.shape, crisis_data.shape

In [ ]:

res_dict_regular = full_evaluation(regular_fake_data, regular_data, eval_config)
# for k, v in res_dict_regular.items():
#     print(k, v)

res_dict_crisis = full_evaluation(crisis_fake_data, crisis_data, eval_config)
# for k, v in res_dict_regular.items():
#     print(k, v)

In [ ]:
df_result = pd.DataFrame.from_dict(res_dict_regular, orient='index')
df_result.columns = ['Regular']
df_result['Crisis'] = res_dict_crisis
df_result['Score'] = 0.7*df_result['Crisis'] + 0.3*df_result['Regular']

df_result.round(5)

In [ ]:
auc = compute_auc(crisis_data, crisis_fake_data, regular_data, regular_fake_data, eval_config)
print(auc)